In [9]:
import pyarrow as pa
import pyarrow.fs
import io
import requests

In [10]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 24883167232 (23.17 GB)
Present Capacity: 8085766467 (7.53 GB)
DFS Remaining: 7815376896 (7.28 GB)
DFS Used: 270389571 (257.86 MB)
DFS Used%: 3.34%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 78
	Missing blocks (with replication factor 1): 78
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.20.0.5:9866 (p4_wadadkar-dn-2.p4_wadadkar_default)
Hostname: d529d9fd4a2d
Decommission Status : Normal
Configured Capacity: 24883167232 (23.17 GB)
DFS Used: 270389571 (257.86 MB)
Non DFS Used: 16780623549 (15.63 GB)
DFS Remaining: 7815376896 (7.28 GB)
DFS Used%: 1.09%
DFS Remaining%: 31.41%


In [2]:
import time
time.sleep(30)

In [11]:
#q9

# Define the WebHDFS URL for accessing single.csv
file_url = "http://boss:9870/webhdfs/v1/single.csv"
params = {
    "op": "GETFILESTATUS"
}

# Retrieve file size and block size
file_status = requests.get(file_url, params=params).json()
file_size = file_status["FileStatus"]["length"]
block_size = file_status["FileStatus"]["blockSize"]
num_blocks = (file_size + block_size - 1) // block_size  # Calculate the total number of blocks

# Dictionary to store block distribution and a list for accessible blocks
block_distribution = {
    "lost": 0,
}
accessible_blocks = []

# Loop through each block to check its DataNode location
for block_num in range(num_blocks):
    offset = block_num * block_size
    params = {
        "op": "OPEN",
        "offset": offset,
        "noredirect": "true"
    }
    
    # Request block location
    response = requests.get(file_url, params=params)
    
    # If the block is inaccessible, count it as "lost"
    if response.status_code == 403:
        block_distribution["lost"] += 1
    elif response.status_code == 200:
        # Extract the container ID from the Location header if available
        location = response.json().get("Location")
        container_id = location.split("//")[1].split(":")[0]

        if container_id in block_distribution:
            block_distribution[container_id] += 1
        else:
            block_distribution[container_id] = 1
        accessible_blocks.append(offset)
block_distribution

{'lost': 78, 'd529d9fd4a2d': 89}

In [12]:
#q10

# Connect to HDFS on the Namenode at port 9000
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
file_path = "/single.csv"

# Initialize count for occurrences of "Single Family"
single_family_count = 0

# Loop through each accessible block's offset
for offset in accessible_blocks:
    with hdfs.open_input_file(file_path) as file:
        # Read only the accessible block's content
        block_content = file.read_at(block_size, offset).decode('utf-8')

        # Use io.StringIO to treat the block content as a text stream
        text_stream = io.StringIO(block_content)
        
        # Count lines containing "Single Family" within this block
        single_family_count += sum(1 for line in text_stream if "Single Family" in line)

# Output the total count of "Single Family" in accessible blocks
single_family_count

2024-11-15 06:44:23,925 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


237366